# The dataset

'50 startups'. Different startups with variables:

* Profit
* R&D Spend
* Administration
* Marketing Spend
* State - Categorical variable!

What yields best results? Which companies are more interesting?

# Regressions

* Simple linear: One independent
* Multiple linear: Several independent

Assumptions:

* Linearity: Data is actually linear
* Homoscedasticity: Same finite variance - variance homogenous even for larger values.
* Multivariate normality: Aha, normal distribution in multiple dimensions when combined. (Can it be different if sub-components are normal?)
* Independence of errors: Errors are not systematic?
* Lack of multicollinearity: Lines not codependent? No interaction?


# Dummy variables

Dummy variable: Replacing New York / California with a 0/1 value.

If using multiple dummy variable levels - if you retain both it messes things up. You always include one less than levels. 

The risk: It leads to situation where one variable can be predicted using another.

http://www.algosome.com/articles/dummy-variable-trap-regression.html



# What is a p

https://www.mathbootcamps.com/what-is-a-p-value/

`P-value tells us how unlikely result is if hypothesis is true`.

p = 0.18 here tells us that probability of getting a mean of 68.7 or less from sample of particular size is 0.18 if the peanuts in the population is 70 grams or more.

# How to build models - Step by step

Framework for several methods. All columns potential predictors! Why do we want to trim our model?

* Garbage in garbage out
* Intuition - Need to be able to explain what's up

## All-in

Throw in all variables. When:

* We have prior knowledge - Everything is valuable
* Framework saying you have to use everything
* Preparing for backward elimination

## Backward Elimination (stepwise)

Steps:

* Select significance level to stay in model
* Fit full model with *all* possible predictors
* Consider predictor with *highest* value. If > threshold.
* If so, remove the predictor
* Re-fit the model without the variable

## Forward Selection (stepwise)

More complex than just reversing previous.

* Select significance level
* Fit all simple regression models `y ~ x_n`. Select with lowest p.
* Keep this and fit all possible models with one extra predictor. Which additional is best?
* Consider new with lowest p. Lower than threshold, repeat from 3.

## Bidirectional Elimination (stepwise, or Stepwise)

* Select level to enter and to stay
* To enter, need to be less than SLENTER
* Perform all backward elimination. Back to 2.
* No new variables can enter and no variables can exit.

## All possible models

* Select criterion of goodness of fit
* Construct all possible regression models
* Select the one with the best criterion

Can be very big!

In [6]:
library(caTools)
set.seed(123)

source("~/src/jupyterrutils/VisualizationUtils.R")

[1] "Loading module to 'visutil' and 'vu'"


In [11]:
fp <- "../machine_learning_template_folder/Part 2 - Regression/Section 5 - Multiple Linear Regression/Multiple_Linear_Regression/50_Startups.csv"
df <- read.csv(fp)
head(df)

df$State <- factor(df$State)
str(df$State)

split <- sample.split(df$Profit, SplitRatio = 0.8)
training_set <- subset(df, split==T)
test_set <- subset(df, split==F)

dim(df)
dim(training_set)
dim(test_set)

R.D.Spend,Administration,Marketing.Spend,State,Profit
165349.2,136897.80,471784.1,New York,192261.8
162597.7,151377.59,443898.5,California,191792.1
153441.5,101145.55,407934.5,Florida,191050.4
144372.4,118671.85,383199.6,New York,182902.0
142107.3,91391.77,366168.4,Florida,166187.9
131876.9,99814.71,362861.4,New York,156991.1


 Factor w/ 3 levels "California","Florida",..: 3 1 2 3 2 3 1 2 3 1 ...


[1] 50  5

[1] 40  5

[1] 10  5

Feature scaling handled by the regression model, so we don't need this as separate step